# Sprawozdanie 2
## Jakub Ciągło (275986), Mateusz Ćwiek (276011)

___
# Importy

In [40]:
import scipy.stats as stats
import numpy as np
import pandas as pd
from statsmodels.stats.proportion import proportion_confint
import math

___
# Część 1
## Zadanie 1
**Oszacowanie rozkładu opinii o szkoleniu – przedziały ufności dla wektora prawdopodobieństw** 

W zadaniu analizujemy odpowiedzi 200 pracowników, którzy oceniali szkolenie „Efektywna komunikacja w zespole”. Opinie zostały pogrupowane w pięć kategorii:  

Na podstawie uzyskanych liczebności wyznaczamy przedziały ufności dla częstości odpowiedzi w każdej kategorii, traktując te częstości jako estymacje elementów wektora prawdopodobieństw dyskretnego rozkładu opinii.  

Uwzględniamy dwa podejścia:  
- Dokładne przedziały Cloppera-Pearsona oparte na rozkładzie beta,  
- Przybliżone przedziały Wilsona (asymptotyczne).  

Ze względu na pięć kategorii, poziom ufności dla każdej z nich został skorygowany metodą Bonferroniego (czyli poziom istotności $ \alpha = 0.05 $ podzielono przez $5$).  

Przedziały zostały wyznaczone niezależnie dla każdej kategorii, zakładając próbę o stałej liczebności $ n = 200 $.

In [41]:
def clopper_pearson_ci(successes, trials, alpha):
    lower = stats.beta.ppf(alpha / 2, successes, trials - successes + 1)
    upper = stats.beta.ppf(1 - alpha / 2, successes + 1, trials - successes)
    return lower, upper

def wyznacz_przedzialy_ufnosci(liczebnosci, alpha=0.05):
    n = np.sum(liczebnosci)
    kategorie = len(liczebnosci)
    
    adjusted_alpha = alpha / kategorie

    przedzialy = []

    for i, sukcesy in enumerate(liczebnosci):
        cp_lower, cp_upper = clopper_pearson_ci(sukcesy, n, adjusted_alpha)

        wilson_lower, wilson_upper = proportion_confint(sukcesy, n, alpha=adjusted_alpha, method='wilson')

        przedzialy.append({
            'kategoria': i + 1,
            'dokladny': (cp_lower, cp_upper),
            'asymptotyczny': (wilson_lower, wilson_upper)
        })

    return przedzialy

liczebnosci = np.array([14, 17, 40, 100, 29])
przedzialy = wyznacz_przedzialy_ufnosci(liczebnosci, alpha=0.05)

for przedzial in przedzialy:
    print(f"Kategoria {przedzial['kategoria']}")
    print(f"  Metoda Clopper-Pearson: ({przedzial['dokladny'][0]:.3f}, {przedzial['dokladny'][1]:.3f})")
    print(f"  Metoda Wilson:          ({przedzial['asymptotyczny'][0]:.3f}, {przedzial['asymptotyczny'][1]:.3f})\n")


Kategoria 1
  Metoda Clopper-Pearson: (0.032, 0.130)
  Metoda Wilson:          (0.036, 0.132)

Kategoria 2
  Metoda Clopper-Pearson: (0.042, 0.149)
  Metoda Wilson:          (0.047, 0.150)

Kategoria 3
  Metoda Clopper-Pearson: (0.133, 0.282)
  Metoda Wilson:          (0.137, 0.282)

Kategoria 4
  Metoda Clopper-Pearson: (0.407, 0.593)
  Metoda Wilson:          (0.410, 0.590)

Kategoria 5
  Metoda Clopper-Pearson: (0.087, 0.220)
  Metoda Wilson:          (0.092, 0.221)



___
## Zadanie 2
**Poziomy krytyczne w testach dopasowania rozkładu – testy chi-kwadrat**

Celem zadania było stworzenie funkcji obliczającej poziomy krytyczne (p-value) w dwóch klasycznych testach dopasowania rozkładu wielomianowego:

- Test chi-kwadrat Pearsona,  
- Test chi-kwadrat największej wiarygodności.

Weryfikujemy hipotezę:
$$
H_0: \mathbf{p} = \mathbf{p}_0 \quad \text{przeciwko} \quad H_1: \mathbf{p} \ne \mathbf{p}_0
$$
na podstawie obserwacji $\mathbf{x} = (x_1, \dots, x_k)$ będącej realizacją zmiennej losowej z rozkładu wielomianowego o parametrach $n$ (łączna liczba prób) i $\mathbf{p}$.

W funkcji:

- Obliczono wartość statystyki testowej Pearsona:  
  $$
  \chi^2 = \sum_{i=1}^k \frac{(x_i - np_{0i})^2}{np_{0i}}
  $$
- Obliczono statystykę testu największej wiarygodności:  
  $$
  G^2 = 2 \sum_{i=1}^k x_i \log \left( \frac{x_i}{np_{0i}} \right)
  $$
- Dla obu statystyk zwrócono wartość p-value obliczoną na podstawie rozkładu $\chi^2$ z $k - 1$ stopniami swobody.  

Funkcja przyjmuje dowolny rozkład teoretyczny $\mathbf{p}_0$ i zestaw danych $\mathbf{x}$, co pozwala łatwo porównywać dane empiryczne z dowolną hipotezą dotyczącą struktury rozkładu.

In [42]:
def poziomy_krytyczne(x, p_0):
    k = len(p_0)
    n = sum(x)
    
    pearson_stat = sum(((x[i] - n * p_0[i]) ** 2) / (n * p_0[i]) for i in range(k))

    nw_stat = 2 * sum(x[i] * np.log(x[i] / (n * p_0[i])) for i in range(k) if x[i] > 0)
    
    p_value_pearson = 1 - stats.chi2.cdf(pearson_stat, k - 1)
    p_value_nw = 1 - stats.chi2.cdf(nw_stat, k - 1)
    
    return float(p_value_pearson), float(p_value_nw)

___
## Zadanie 3

**Test zgodności rozkładu odpowiedzi z rozkładem jednostajnym — Dział Produktowy**

W tym zadaniu sprawdzono, czy rozkład odpowiedzi pracowników z Działu Produktowego na pytanie dotyczące wsparcia i materiałów szkoleniowych (PYT_1) można uznać za jednostajny, czyli czy każde z pięciu możliwych stanowisk (-2, -1, 0, 1, 2) było wybierane z jednakowym prawdopodobieństwem.

Do weryfikacji hipotezy:
$$
H_0: \text{rozkład odpowiedzi jest jednostajny (równe prawdopodobieństwa)} \\
H_1: \text{rozkład odpowiedzi nie jest jednostajny}
$$
wykorzystano dwie statystyki chi-kwadrat:

- **Test Pearsona**,  
- **Test największej wiarygodności**.

Na podstawie otrzymanych wartości p (p-value) z obu testów oceniono, czy istnieją podstawy do odrzucenia hipotezy zerowej na poziomie istotności $\alpha = 0.05$.  

In [43]:
alfa = 0.05
df = pd.read_csv('ankieta.csv', encoding='Latin2', sep=';')

df_produktowy = df[df['DZIAŁ'] == 'PD']

odpowiedzi = [-2, -1, 0, 1, 2]
n = df_produktowy['PYT_1'].value_counts().reindex(odpowiedzi, fill_value=0).tolist()
print('Liczności odpowiedzi:', n)

p_0 = [1/len(n) for _ in range(len(n))] # w hipotezie zakładamy, że prawdopodobieństwa są równe
p_value_pearson, p_value_nw = poziomy_krytyczne(n,p_0)

p_value_pearson, p_value_nw = poziomy_krytyczne(n, p_0)

print(f'Statystyka chi² Pearsona (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_pearson}')
if p_value_pearson > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

print(f'\nStatystyka chi² największej wiarygodności (dla poziomu istotności α = {alfa}):')
print(f'  p-wartość = {p_value_nw}')
if p_value_nw > alfa:
    print('  → Brak podstaw do odrzucenia hipotezy zerowej (rozkład może być równomierny)')
else:
    print('  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)')

Liczności odpowiedzi: [9, 10, 17, 51, 11]
Statystyka chi² Pearsona (dla poziomu istotności α = 0.05):
  p-wartość = 2.757793993168889e-13
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)

Statystyka chi² największej wiarygodności (dla poziomu istotności α = 0.05):
  p-wartość = 1.0701994845874196e-10
  → Odrzucamy hipotezę zerową (rozkład nie jest równomierny)


___
## Zadanie 5

In [ ]:
# 2. Budowanie tabeli 2×2
contingency_table = pd.crosstab(df['PŁEĆ'], df['CZY_KIER'])
print(contingency_table)

oddsratio, p_value = stats.fisher_exact(contingency_table)

print("Wynik testu Fishera:")
print(f"  odds ratio: {oddsratio}")
print(f"  p-value: {p_value}")

alpha = 0.05
if p_value < alpha:
    print("Odrzucamy hipotezę zerową -> zmienne 'PŁEĆ' i 'CZY_KIER' są zależne.")
else:
    print("Brak podstaw do odrzucenia H0 -> nie ma dowodów na zależność.")


CZY_KIER  Nie  Tak
PŁEĆ              
K          63    8
M         110   19
Wynik testu Fishera:
  odds ratio: 1.3602272727272726
  p-value: 0.6659028889666552
Brak podstaw do odrzucenia H0 -> nie ma dowodów na zależność.


Jeśli p‑value < 0.05, odrzucamy hipotezę o niezależności, co oznacza, że istnieje istotna statystycznie różnica w prawdopodobieństwie bycia na stanowisku kierowniczym między kobietami a mężczyznami.

Jeśli p‑value ≥ 0.05 - tak jak w wyniku zadania, mamy brak podstaw do odrzucenia hipotezy zerowej, czyli test Fishera ($p = 0.6659$) nie wykazał istotnej różnicy między kobietami i mężczyznami w udziale stanowisk kierowniczych, co oznacza, że prawdopodobieństwo pełnienia funkcji kierowniczej jest w obu grupach statystycznie zbliżone.

___
## Zadanie 6

In [45]:
import os
os.environ["R_HOME"] = r"C:\Program Files\R\R-4.5.0"
os.environ["PATH"] += r";C:\Program Files\R\R-4.5.0\bin\x64"
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
stats = importr("stats") 

In [46]:
from rpy2.robjects import pandas2ri; pandas2ri.activate()

df["CZY_ZADOW"] = df["PYT_2"].replace({
    -2: "niezadowolony",
    -1: "niezadowolony",
     1: "zadowolony",
     2: "zadowolony"
})

df["WIEK_KAT"] = pd.cut(
        df["WIEK"],
        bins=[0, 35, 45, 55, float("inf")],
        labels=["do 35 lat", "36‑45 lat", "46‑55 lat", "powyżej 55 lat"])


def fisher_fh(tab, simulate=False, B=50000, workspace=300000):
    r_tab = pandas2ri.py2rpy(tab)
    res   = stats.fisher_test(
               r_tab,
               simulate_p_value = simulate,
               B        = B if simulate else 0,
               workspace = workspace if not simulate else None
           )
    return dict(res.items())["p.value"][0]


hipotezy = [
    ("CZY_KIER", "WIEK_KAT",  "a"),
    ("CZY_KIER", "STAŻ",    "b"),
    ("PYT_2",    "CZY_KIER",  "c"),
    ("PYT_2",    "STAŻ",    "d"),
    ("PYT_2",    "PŁEĆ",      "e"),
    ("PYT_2",    "WIEK_KAT",  "f"),
]

for x, y, et in hipotezy:
    tab = pd.crosstab(df[x], df[y])
    p   = fisher_fh(tab, simulate = tab.size > 25)
    decyzja = "ODRZUCAMY H₀" if p < 0.05 else "brak podstaw"
    print(f"{et}) {x} × {y:<9}  p = {p:6.4f}  → {decyzja}")

print("\n--- PYT_2 zastąpione przez CZY_ZADOW ---")
for x, y, et in [
        ("CZY_ZADOW", "CZY_KIER", "c′"),
        ("CZY_ZADOW", "STAŻ",   "d′"),
        ("CZY_ZADOW", "PŁEĆ",     "e′"),
        ("CZY_ZADOW", "WIEK_KAT", "f′")]:
    tab = pd.crosstab(df[x], df[y])
    p   = fisher_fh(tab, simulate = tab.size > 25)
    decyzja = "ODRZUCAMY H₀" if p < 0.05 else "brak podstaw"
    print(f"{et}) {x} × {y:<9}  p = {p:6.4f}  → {decyzja}")


a) CZY_KIER × WIEK_KAT   p = 0.7823  → brak podstaw
b) CZY_KIER × STAŻ       p = 0.0001  → ODRZUCAMY H₀
c) PYT_2 × CZY_KIER   p = 0.0443  → ODRZUCAMY H₀
d) PYT_2 × STAŻ       p = 0.0107  → ODRZUCAMY H₀
e) PYT_2 × PŁEĆ       p = 0.4758  → brak podstaw
f) PYT_2 × WIEK_KAT   p = 0.3194  → brak podstaw

--- PYT_2 zastąpione przez CZY_ZADOW ---
c′) CZY_ZADOW × CZY_KIER   p = 0.8377  → brak podstaw
d′) CZY_ZADOW × STAŻ       p = 0.4097  → brak podstaw
e′) CZY_ZADOW × PŁEĆ       p = 0.6589  → brak podstaw
f′) CZY_ZADOW × WIEK_KAT   p = 0.3275  → brak podstaw


W świetle przeprowadzonych testów okazuje się, że zajmowanie stanowiska kierowniczego nie ma związku z wiekiem ($p = 0.7823$), natomiast jest silnie związane ze stażem pracy ($p = 0,0001$). Gdy używamy oryginalnej, wielostopniowej skali satysfakcji (PYT_2), to poziom zadowolenia okazuje się zależeć zarówno od pełnienia funkcji kierowniczej ($p = 0.0443$), jak i od stażu pracy ($p = 0.0107$), podczas gdy płeć i wiek nie wykazują wpływu. Jednak po zredukowaniu skali do dwuwartościowej zmiennej CZY_ZADOW wszystkie testy c′–f′ tracą istotność (najniższe $p = 0.3275$), co sugeruje, że agregacja odpowiedzi z tytułu uproszczenia skali wygładza różnice i „maskuje” wcześniej obserwowane zależności.